# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

To simplify our analysis, we're going to subset the data since we (initially) only need the `PATENT` and `POSTATE` data. This will make it easier to read the output of the various joins we'll do below.

In [8]:
patState = patents.filter(patents.COUNTRY == 'US').select(['PATENT','POSTATE']).cache()

Now, we're going to do our first join. We'll use [and inner join](http://www.sql-join.com/sql-join-types) because we only want entries where both the patent is in the database and it actually cites some other patent. Following that, we're going to the drop the `PATENT` column because we alredy have that information in the `CITING` column. We're also going to rename the `POSTATE` column to `CITING_STATE` because we're going to join this table again and at that time we would have two `POSTATE` columns -- better to disambiguate now.

In [9]:
byCiting = patState.join(citations,patState.PATENT==citations.CITING, how='inner')\
           .drop("PATENT").withColumnRenamed('POSTATE', 'CITING_STATE')

Our `byCiting` dataframe should now having the `CITING_STATE` for the `CITING` column, and only for the patents that are in the patent database and in the citations.

In [10]:
byCiting.show(5)

+------------+-------+-------+
|CITING_STATE| CITING|  CITED|
+------------+-------+-------+
|          CA|3858258|1331793|
|          CA|3858258|1540798|
|          IN|3858560| 957631|
|          MT|3858597|3675252|
|          MT|3858597|3815160|
+------------+-------+-------+
only showing top 5 rows



Now we do our second join on the patent data `PATENT` and the `CITED` column to determine the state for each cited patent. And, again, we rename columns etc.

In [11]:
byCited = patState.join(byCiting, patState.PATENT == byCiting.CITED, how='inner')\
          .drop("PATENT").withColumnRenamed('POSTATE', 'CITED_STATE')

In [12]:
byCited.show(5)

+-----------+------------+-------+-------+
|CITED_STATE|CITING_STATE| CITING|  CITED|
+-----------+------------+-------+-------+
|         MN|          NJ|4496943|3071753|
|         MN|          TN|4345315|3071753|
|         MN|          IL|4120573|3071753|
|         MN|          NJ|3949375|3071753|
|         MN|          NY|4271479|3071753|
+-----------+------------+-------+-------+
only showing top 5 rows



At this point, we can eliminate all rows of this table where the stated of the citing and cited patent aren't the same.

In [13]:
sameState = byCited.filter( byCited.CITING_STATE == byCited.CITING_STATE )

Now, are table contains multiple entries for each `CITING` patent where the `CITED` entry is in the same state. We can count up the `CITED` entries using the `groupBy` mechanism and then [use the aggreation operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct) to count up the number of `CITED`. The column name of the number of same state patents is ugly, so I rename it to `SAME_STATE`.

In [14]:
sameStateCount = sameState.groupBy('CITING').agg({"CITED" : "count"})\
                .withColumnRenamed('count(CITED)', 'SAME_STATE')

In [15]:
sameStateCount.show(5)

+-------+----------+
| CITING|SAME_STATE|
+-------+----------+
|3858597|         2|
|3859029|         2|
|3859627|         5|
|3860038|         3|
|3860100|         3|
+-------+----------+
only showing top 5 rows



Now, we need to produce the new patent data by joining the "same state" count into the original patent data. This involves a *left join* because we want to retain all the patents, including those that don't have any `SAME_STATE` references. We also drop the `CITING` column because it duplicates the `PATENT` column.

In [16]:
newPatentData = patents.join(sameStateCount, patents.PATENT==sameStateCount.CITING, how='left').drop('CITING').cache()

In [17]:
newPatentData.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|3070853| 1963| 1096|   null|     US|     FL|    null|      1|  null|    49|  5|    59| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|      null|
|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|      null|
|3071452| 1963| 1096|   null| 

We could now write out the data to a (collection of) CSV files. In my example, this produced ~200 files of data. I would not recommend storing that.

In [18]:
#newPatentData.write.csv("patent-data-augmented.csv")

Now, the final output -- the 10 patents with the largest number of same-state citations. The only thing that is complex here is that the PySpark value of "null" complicates sorting by a numeric field -- thus, we filter out the null entries and then sort by the `SAME_STATE`

In [19]:
newPatentData.filter(newPatentData.SAME_STATE.isNotNull()).sort('SAME_STATE', ascending=False).show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5887243| 1999|14326|   1995|     US|     NY|  752434|      2|  null|   455|  2|    21|  745|       0|  0.9785|   null|  0.8195|    null| 16.9289|     0.0|     0.0|    null|    null|       524|
|5795784| 1998|14109|   1996|     US|     TX|     735|      2|     3|   436|  1|    19|  770|       0|   0.987|   null|  0.8489|    null| 13.0312|  0.0274|  0.0234|    null|    null|       495|
|5856194| 1999|14249|   1996| 